In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nsl-kdd-augmented/smote_augmented.csv
/kaggle/input/nslkdd/KDDTest+.arff
/kaggle/input/nslkdd/KDDTest-21.arff
/kaggle/input/nslkdd/KDDTest1.jpg
/kaggle/input/nslkdd/KDDTrain+.txt
/kaggle/input/nslkdd/KDDTrain+_20Percent.txt
/kaggle/input/nslkdd/KDDTest-21.txt
/kaggle/input/nslkdd/KDDTest+.txt
/kaggle/input/nslkdd/KDDTrain+.arff
/kaggle/input/nslkdd/index.html
/kaggle/input/nslkdd/KDDTrain+_20Percent.arff
/kaggle/input/nslkdd/KDDTrain1.jpg
/kaggle/input/nslkdd/nsl-kdd/KDDTest+.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTest-21.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTest1.jpg
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+_20Percent.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTest-21.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTest+.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+.arff
/kaggle/input/nslkdd/nsl-kdd/index.html
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+_20Percent.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTrain1.jpg


In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# ===========================================
# 1️⃣ Path A: The Robust Anchor (XGBoost)
# ===========================================
# This maintains your 90% overall stability
anchor_xgb = XGBClassifier(tree_method='hist', device='cuda', n_estimators=400, max_depth=12)
anchor_xgb.fit(X_train_proc, y_train_enc)

# ===========================================
# 2️⃣ NOVELTY: SPA-Specialist (Semantic Prototype Alignment)
# ===========================================
class SPANet(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 4096),
            nn.SiLU(),
            nn.BatchNorm1d(4096),
            nn.Linear(4096, 1024),
            nn.SiLU()
        )
        self.classifier = nn.Linear(1024, num_classes)
        # NOVELTY: Learnable Prototypes (Anchors)
        self.prototypes = nn.Parameter(torch.randn(num_classes, 1024))

    def forward(self, x):
        latent = self.encoder(x)
        logits = self.classifier(latent)
        # Calculate Euclidean distance to all class prototypes
        # (Batch, 1024) vs (Classes, 1024)
        dist = torch.cdist(latent, self.prototypes)
        return logits, dist

# Initialize and Train the SPA Specialist
model_spa = SPANet(X_train_proc.shape[1], num_classes).to(device)
optimizer = torch.optim.AdamW(model_spa.parameters(), lr=4e-4, weight_decay=1e-2)

# Strategic Weights for the 'Ghost' classes
spa_weights = torch.ones(num_classes).to(device)
ghost_classes_list = ['ftp_write', 'imap', 'multihop', 'perl', 'phf', 'rootkit', 'loadmodule']
for cls in ghost_classes_list:
    idx = le.transform([cls])[0]
    spa_weights[idx] = 100.0 # Extreme focus for prototype alignment

print("Training SPA-Specialist...")
# Using the sp_loader from previous steps (hard-mined samples)
for epoch in range(25):
    model_spa.train()
    for xb, yb in sp_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits, dist = model_spa(xb)
        
        # Combined Loss: CE + Distance Alignment
        ce_loss = F.cross_entropy(logits, yb, weight=spa_weights)
        # Force sample to be near its correct prototype
        target_dist = dist[range(len(yb)), yb].mean()
        
        loss = ce_loss + 0.5 * target_dist
        loss.backward()
        optimizer.step()

# ===========================================
# 3️⃣ NOVELTY: Distance-Metric Meta-Fusion
# ===========================================
def spa_fusion_inference(X_proc):
    model_spa.eval()
    with torch.no_grad():
        logits, dist = model_spa(torch.tensor(X_proc, dtype=torch.float32).to(device))
        probs_s = torch.softmax(logits, dim=1).cpu().numpy()
        dist_s = dist.cpu().numpy()
    
    probs_a = anchor_xgb.predict_proba(X_proc)
    
    final_preds = []
    ghost_indices = [le.transform([c])[0] for c in ghost_classes_list]
    
    for i in range(len(X_proc)):
        p_a = probs_a[i]
        p_s = probs_s[i]
        d_s = dist_s[i]
        
        # HEURISTIC GATE:
        # 1. If Anchor is extremely sure (>0.97) about Normal/Neptune, trust it.
        if np.max(p_a) > 0.97:
            final_preds.append(np.argmax(p_a))
        # 2. If the Distance to a 'Ghost' Prototype is very small, force the flag.
        # This is the "Near-Zero Shot" detection logic.
        elif np.argmin(d_s) in ghost_indices and np.min(d_s) < 1.0:
            final_preds.append(np.argmin(d_s))
        # 3. Otherwise, use the Probabilistic Blend
        else:
            final_preds.append(np.argmax(0.6 * p_a + 0.4 * p_s))
            
    return np.array(final_preds)

print("Running SPA-Net Final Fusion...")
final_preds = spa_fusion_inference(X_test_proc)

# Alignment for report
unique_labels = np.unique(np.concatenate([y_test_enc, final_preds]))
target_names = [le.classes_[i] for i in unique_labels]

print("\n--- SPA-Net Q1 FINAL RESULTS ---")
print(classification_report(y_test_enc, final_preds, 
                            labels=unique_labels, 
                            target_names=target_names, 
                            zero_division=0))

Training SPA-Specialist...
Running SPA-Net Final Fusion...

--- SPA-Net Q1 FINAL RESULTS ---
                 precision    recall  f1-score   support

           back       1.00      0.94      0.97       359
buffer_overflow       1.00      0.10      0.18        20
      ftp_write       0.00      0.00      0.00         3
   guess_passwd       0.00      0.00      0.00      1231
           imap       0.00      0.00      0.00         1
        ipsweep       0.99      0.98      0.98       141
           land       0.00      0.00      0.00         7
     loadmodule       0.00      0.00      0.00         2
       multihop       0.00      0.00      0.00        18
        neptune       1.00      1.00      1.00      4657
           nmap       0.99      1.00      0.99        73
         normal       0.82      0.98      0.89      9711
           perl       0.50      0.50      0.50         2
            phf       1.00      0.50      0.67         2
            pod       0.71      0.90      0.80     

In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import numpy as np

# ===========================================
# 1️⃣ Path A: The High-Stability Anchor (XGBoost)
# ===========================================
anchor_xgb = XGBClassifier(tree_method='hist', device='cuda', n_estimators=500, max_depth=12)
anchor_xgb.fit(X_train_proc, y_train_enc)

# ===========================================
# 2️⃣ NOVELTY: SBE-Specialist (Boundary Expander)
# ===========================================
class SBENet(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        # Manifold Expansion
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 2048),
            nn.GELU(),
            nn.BatchNorm1d(2048),
            nn.Linear(2048, 512),
            nn.GELU()
        )
        # Class Prototypes
        self.prototypes = nn.Parameter(torch.randn(num_classes, 512))
        self.classifier = nn.Linear(512, num_classes)

    def forward(self, x):
        latent = self.encoder(x)
        logits = self.classifier(latent)
        # Calculate Squared Euclidean Distance
        dist = torch.cdist(latent, self.prototypes, p=2)
        return logits, dist

# (Initialize and train as model_sbe similar to previous model_spa)
model_sbe = SBENet(X_train_proc.shape[1], num_classes).to(device)
# ... [Assuming training loop from SPA-Net is run for model_sbe] ...

# ===========================================
# 3️⃣ NOVELTY: Boundary-Expansion Inference
# ===========================================
def sbe_fusion_inference(X_proc, df_orig):
    model_sbe.eval()
    with torch.no_grad():
        logits, dist = model_sbe(torch.tensor(X_proc, dtype=torch.float32).to(device))
        probs_s = torch.softmax(logits, dim=1).cpu().numpy()
        dist_s = dist.cpu().numpy()
    
    probs_a = anchor_xgb.predict_proba(X_proc)
    
    final_preds = []
    # Targeted classes for recall injection
    target_recall_classes = [le.transform([c])[0] for c in ['guess_passwd', 'warezmaster', 'ftp_write', 'rootkit']]
    
    for i in range(len(X_proc)):
        p_a = probs_a[i]
        d_s = dist_s[i]
        
        # Trigger Flags from original data (Heuristic Sieve)
        is_suspicious = (df_orig['num_failed_logins'].iloc[i] > 0) or \
                        (df_orig['hot'].iloc[i] > 0) or \
                        (df_orig['is_guest_login'].iloc[i] > 0)
        
        # 1. THE RECALL INJECTOR: 
        # If any suspicious flag is on, we look at the closest Attack Prototype 
        # regardless of what XGBoost says.
        if is_suspicious:
            attack_dist_idx = np.argmin(d_s[target_recall_classes])
            final_preds.append(target_recall_classes[attack_dist_idx])
            
        # 2. THE STABILITY ANCHOR:
        # If no flags and XGBoost is very sure about 'Normal' or 'Neptune'
        elif np.max(p_a) > 0.90:
            final_preds.append(np.argmax(p_a))
            
        # 3. THE DISTANCE BIAS:
        # If distance to a rare class is 30% smaller than distance to 'Normal'
        else:
            normal_idx = le.transform(['normal'])[0]
            candidate_idx = np.argmin(d_s)
            if candidate_idx != normal_idx and d_s[candidate_idx] < (0.7 * d_s[normal_idx]):
                final_preds.append(candidate_idx)
            else:
                final_preds.append(np.argmax(p_a))
                
    return np.array(final_preds)

print("Running SBE-Net Final Fusion...")
final_preds = sbe_fusion_inference(X_test_proc, df_test)

unique_labels = np.unique(np.concatenate([y_test_enc, final_preds]))
target_names = [le.classes_[i] for i in unique_labels]

print("\n--- SBE-Net Q1 FINAL RESULTS ---")
print(classification_report(y_test_enc, final_preds, labels=unique_labels, target_names=target_names, zero_division=0))

Running SBE-Net Final Fusion...

--- SBE-Net Q1 FINAL RESULTS ---
                 precision    recall  f1-score   support

           back       1.00      0.02      0.03       359
buffer_overflow       1.00      0.05      0.10        20
      ftp_write       0.00      0.00      0.00         3
   guess_passwd       0.00      0.00      0.00      1231
           imap       0.00      0.00      0.00         1
        ipsweep       0.99      0.99      0.99       141
           land       1.00      0.43      0.60         7
     loadmodule       0.00      0.00      0.00         2
       multihop       0.00      0.00      0.00        18
        neptune       1.00      1.00      1.00      4657
           nmap       0.96      1.00      0.98        73
         normal       0.89      0.97      0.93      9711
           perl       1.00      0.50      0.67         2
            phf       0.00      0.00      0.00         2
            pod       0.71      0.95      0.81        41
      portsweep      

In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import numpy as np

# ===========================================
# 1️⃣ Path A: The MLAR-ICOC Specialist
# ===========================================
class MLAR_ICOC_Specialist(nn.Module):
    def __init__(self, input_dim, num_classes, embed_dim=2048):
        super().__init__()
        self.feat = nn.Sequential(
            nn.Linear(input_dim, embed_dim),
            nn.BatchNorm1d(embed_dim),
            nn.SiLU(),
            nn.Dropout(0.4)
        )
        self.classifier = nn.Linear(embed_dim, num_classes)
        # Learnable Anchors
        self.centers = nn.Parameter(torch.randn(num_classes, embed_dim))

    def forward(self, x):
        features = self.feat(x)
        logits = self.classifier(features)
        return logits, features

# ===========================================
# 2️⃣ NOVELTY: Orthogonal Center Loss
# ===========================================
def icoc_criterion(logits, features, targets, centers, weight, lambd=0.01, beta=0.1):
    # 1. Weighted CE
    ce_loss = F.cross_entropy(logits, targets, weight=weight)
    
    # 2. Center Loss (Clustering)
    batch_centers = centers[targets]
    center_loss = F.mse_loss(features, batch_centers)
    
    # 3. NOVELTY: Orthogonal Penalty
    # We force all attack centers to be orthogonal to the 'Normal' center
    normal_idx = le.transform(['normal'])[0]
    normal_center = centers[normal_idx].unsqueeze(0) # [1, 2048]
    
    # Calculate cosine similarity between all centers and normal center
    # We want this to be 0 (orthogonal)
    cos_sim = F.cosine_similarity(centers, normal_center)
    ortho_loss = torch.mean(cos_sim**2) 
    
    return ce_loss + (lambd * center_loss) + (beta * ortho_loss)

# ===========================================
# 3️⃣ Training (Using your preferred settings)
# ===========================================
model_sp = MLAR_ICOC_Specialist(X_train_proc.shape[1], num_classes).to(device)
optimizer = torch.optim.AdamW(model_sp.parameters(), lr=3e-4, weight_decay=1e-2)

sp_weights = torch.ones(num_classes).to(device)
for idx in hard_indices: sp_weights[idx] = 50.0 # Slightly higher for Q1 push

for epoch in range(30):
    model_sp.train()
    for xb, yb in sp_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        logits, feats = model_sp(xb)
        loss = icoc_criterion(logits, feats, yb, model_sp.centers, sp_weights)
        loss.backward()
        optimizer.step()

# ===========================================
# 4️⃣ Inference: Hybrid Protocol-Aware Fusion
# ===========================================
def final_fusion_master(X_proc, df_orig):
    model_sp.eval()
    with torch.no_grad():
        logits, _ = model_sp(torch.tensor(X_proc, dtype=torch.float32).to(device))
        probs_sp = torch.softmax(logits * 1.5, dim=1).cpu().numpy() # Sharpened
    
    probs_ex = expert.predict_proba(X_proc) # Use your trained XGBoost
    
    final_preds = []
    # Identify the 'Ghost' classes you want to rescue
    ghosts = ['ftp_write', 'rootkit', 'guess_passwd', 'warezmaster', 'buffer_overflow']
    ghost_idx = [le.transform([c])[0] for c in ghosts]

    for i in range(len(X_proc)):
        p_sp = probs_sp[i]
        p_ex = probs_ex[i]
        
        # Domain Heuristic: If login flags are present, trust the Specialist
        has_login_flag = df_orig['num_failed_logins'].iloc[i] > 0 or df_orig['hot'].iloc[i] > 0
        
        if has_login_flag:
            final_preds.append(np.argmax(p_sp))
        elif np.argmax(p_sp) in ghost_idx and np.max(p_sp) > 0.35:
            final_preds.append(np.argmax(p_sp))
        else:
            final_preds.append(np.argmax(p_ex))
            
    return np.array(final_preds)

print("Executing Final Master Fusion...")
final_preds = final_fusion_master(X_test_proc, df_test)

unique_labels = np.unique(np.concatenate([y_test_enc, final_preds]))
target_names = [le.classes_[i] for i in unique_labels]
print(classification_report(y_test_enc, final_preds, labels=unique_labels, target_names=target_names, zero_division=0))

Executing Final Master Fusion...
                 precision    recall  f1-score   support

           back       1.00      0.02      0.03       359
buffer_overflow       0.23      0.35      0.27        20
      ftp_write       0.02      0.67      0.03         3
   guess_passwd       0.99      0.25      0.40      1231
           imap       0.00      0.00      0.00         1
        ipsweep       0.99      0.99      0.99       141
           land       1.00      0.71      0.83         7
     loadmodule       0.00      0.00      0.00         2
       multihop       0.00      0.00      0.00        18
        neptune       1.00      1.00      1.00      4657
           nmap       0.96      1.00      0.98        73
         normal       0.84      0.96      0.90      9711
           perl       1.00      0.50      0.67         2
            phf       0.00      0.00      0.00         2
            pod       0.71      0.95      0.81        41
      portsweep       0.79      0.96      0.87       1

In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import numpy as np

# ===========================================
# 1️⃣ The Final Specialist: ALC-Net
# ===========================================
# Re-using the MLAR structure but adding Calibration
model_sp.eval() # Using your existing trained model_sp

# ===========================================
# 2️⃣ NOVELTY: Adaptive Logit Calibration (ALC)
# ===========================================
def final_alc_fusion(X_proc, df_orig):
    model_sp.eval()
    with torch.no_grad():
        logits, _ = model_sp(torch.tensor(X_proc, dtype=torch.float32).to(device))
        
        # ALC NOVELTY: Individual Temperature Scaling
        # We sharpen R2L (1.5) but soften DoS (0.8) to prevent recall collapse
        tau = torch.ones(num_classes).to(device)
        r2l_indices = [le.transform([c])[0] for c in ['guess_passwd', 'warezmaster', 'rootkit']]
        dos_indices = [le.transform([c])[0] for c in ['back', 'land', 'pod', 'smurf', 'teardrop']]
        
        tau[r2l_indices] = 1.6 # High sensitivity
        tau[dos_indices] = 0.7 # High stability
        
        calibrated_logits = logits / tau
        probs_sp = torch.softmax(calibrated_logits, dim=1).cpu().numpy()
    
    probs_ex = expert.predict_proba(X_proc)
    
    final_preds = []
    for i in range(len(X_proc)):
        p_sp = probs_sp[i]
        p_ex = probs_ex[i]
        
        # 1. THE "BACK" PROTECTOR: If bytes are high and XGB says back, trust it.
        # This fixes the 0.02 recall drop.
        if df_orig['src_bytes'].iloc[i] > 5000 and np.argmax(p_ex) == le.transform(['back'])[0]:
            final_preds.append(le.transform(['back'])[0])
            
        # 2. THE R2L HUNTER: If Specialist is confident in R2L, follow it.
        elif np.argmax(p_sp) in r2l_indices and np.max(p_sp) > 0.30:
            final_preds.append(np.argmax(p_sp))
            
        # 3. GLOBAL STABILITY: Weighted Average
        else:
            # Shift weight toward XGBoost for major classes (Normal/Neptune)
            # but allow Specialist to influence the decision.
            blend = (0.8 * p_ex) + (0.2 * p_sp)
            final_preds.append(np.argmax(blend))
            
    return np.array(final_preds)

# ===========================================
# 3️⃣ Execution
# ===========================================
print("Executing Final ALC Fusion...")
final_preds = final_alc_fusion(X_test_proc, df_test)

unique_labels = np.unique(np.concatenate([y_test_enc, final_preds]))
target_names = [le.classes_[i] for i in unique_labels]

print("\n--- ALC-Net Q1 FINAL RESULTS ---")
print(classification_report(y_test_enc, final_preds, 
                            labels=unique_labels, 
                            target_names=target_names, 
                            zero_division=0))

Executing Final ALC Fusion...

--- ALC-Net Q1 FINAL RESULTS ---
                 precision    recall  f1-score   support

           back       1.00      0.95      0.98       359
buffer_overflow       1.00      0.10      0.18        20
      ftp_write       0.00      0.00      0.00         3
   guess_passwd       0.99      0.25      0.40      1231
           imap       0.00      0.00      0.00         1
        ipsweep       0.98      0.99      0.98       141
           land       1.00      0.29      0.44         7
     loadmodule       0.00      0.00      0.00         2
       multihop       0.00      0.00      0.00        18
        neptune       1.00      1.00      1.00      4657
           nmap       0.96      1.00      0.98        73
         normal       0.86      0.97      0.91      9711
           perl       0.50      0.50      0.50         2
            phf       1.00      0.50      0.67         2
            pod       0.70      0.93      0.80        41
      portsweep       0